# HR Employee Attrition Analysis - Capstone Part 2 (SQL Analysis)

## Project Overview

In Part 1, we performed comprehensive data analysis using Python and its libraries. Now we shift to SQL-based analysis to gain deeper insights into our HR Analytics dataset using database queries.

## Objectives
1. Create a SQLite3 database from the CSV file
2. Calculate attrition rates by various demographic and employment factors
3. Explore main reasons for attrition using SQL queries
4. Provide actionable insights through effective communication

## Dataset
Continuing with the same HR Employee Attrition dataset used in Part 1.


### Create SQLite3 Database

First, we'll create a SQLite3 database from our CSV file. This allows us to perform SQL queries on our HR dataset efficiently.

In [14]:
# Import required libraries
import sqlite3
import pandas as pd
import numpy as np

# Create connection to SQLite database
conn = sqlite3.connect('hr_attrition.db')

# Read CSV file and load into database
df = pd.read_csv('HR-Employee-Attrition.csv')

# Create table in SQLite database
df.to_sql('employees', conn, if_exists='replace', index=False)

print(f"Database created successfully!")
print(f"Table 'employees' created with {len(df)} records")
print(f"Columns: {len(df.columns)} features")

# Verify table creation
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print(f"Tables in database: {tables}")

# Display first few records to verify
cursor.execute("SELECT * FROM employees LIMIT 5;")
sample_data = cursor.fetchall()
print(f"\nSample data loaded successfully")
print(f"First record: {sample_data[0] if sample_data else 'No data'}")


Database created successfully!
Table 'employees' created with 1470 records
Columns: 35 features
Tables in database: [('employees',)]

Sample data loaded successfully
First record: (41, 'Yes', 'Travel_Rarely', 1102, 'Sales', 1, 2, 'Life Sciences', 1, 1, 2, 'Female', 94, 3, 2, 'Sales Executive', 4, 'Single', 5993, 19479, 8, 'Y', 'Yes', 11, 3, 1, 80, 0, 8, 0, 1, 6, 4, 0, 5)


#### Calculate Attrition Rate

In [16]:
### Calculate Overall Attrition Rate
# SQL query to calculate attrition breakdown
query = """
SELECT 
    ROW_NUMBER() OVER (ORDER BY Attrition DESC) as row,
    Attrition,
    ROUND(
        (COUNT(*) * 100.0 / (SELECT COUNT(*) FROM employees)), 2
    ) as Attrition_rate
FROM employees
GROUP BY Attrition
ORDER BY Attrition DESC;
"""

# Execute query and create dataframe
result = pd.read_sql_query(query, conn)
result

,row,Attrition,Attrition_rate
0,1,Yes,16.12
1,2,No,83.88


### Find Attrition by Gender

Understanding gender-based attrition patterns helps identify if there are any gender-specific retention issues that need to be addressed.


In [19]:
### Implement code

# SQL query to find attrition by gender with specified table structure
query_gender = """
SELECT 
    ROW_NUMBER() OVER (ORDER BY Gender, Attrition) as row,
    Attrition,
    Gender,
    COUNT(*) as Gender_Count,
    ROUND(
        (COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY Gender)), 2
    ) as Attrition_by_gender
FROM employees
GROUP BY Gender, Attrition
ORDER BY Gender, Attrition DESC;
"""

# Execute query and display results
gender_attrition = pd.read_sql_query(query_gender, conn)
gender_attrition

,row,Attrition,Gender,Gender_Count,Attrition_by_gender
0,2,Yes,Female,87,14.80
1,1,No,Female,501,85.20
2,4,Yes,Male,150,17.01
3,3,No,Male,732,82.99


### Find Attrition by Department

Departmental analysis reveals which business units have higher turnover rates, helping to identify department-specific retention challenges.


In [20]:
### Implement code
# SQL query to find attrition by department
query_dept = """
SELECT 
    ROW_NUMBER() OVER (ORDER BY Department, Attrition) as row,
    Department,
    Attrition,
    COUNT(*) as Department_count
FROM employees
GROUP BY Department, Attrition
ORDER BY Department, Attrition DESC;
"""

# Execute query and display results
dept_attrition = pd.read_sql_query(query_dept, conn)
dept_attrition


,row,Department,Attrition,Department_count
0,2,Human Resources,Yes,12
1,1,Human Resources,No,51
2,4,Research & Development,Yes,133
3,3,Research & Development,No,828
4,6,Sales,Yes,92
5,5,Sales,No,354


### Find Attrition by Age Groups

In [23]:
### Implement code
# SQL query to find attrition by age groups
query_age = """
SELECT 
    ROW_NUMBER() OVER (ORDER BY 
        CASE 
            WHEN Age < 30 THEN 'Under 30'
            WHEN Age BETWEEN 30 AND 40 THEN '30-40'
            WHEN Age BETWEEN 40 AND 50 THEN '40-50'
            WHEN Age > 50 THEN 'Over 50'
        END, Attrition DESC) as row,
    Attrition,
    CASE 
        WHEN Age < 30 THEN 'Under 30'
        WHEN Age BETWEEN 30 AND 40 THEN '30-40'
        WHEN Age BETWEEN 40 AND 50 THEN '40-50'
        WHEN Age > 50 THEN 'Over 50'
    END as Age_group,
    COUNT(*) as count,
    ROUND(
        (COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY 
            CASE 
                WHEN Age < 30 THEN 'Under 30'
                WHEN Age BETWEEN 30 AND 40 THEN '30-40'
                WHEN Age BETWEEN 40 AND 50 THEN '40-50'
                WHEN Age > 50 THEN 'Over 50'
            END)), 2
    ) as percent_by_age
FROM employees
GROUP BY 
    CASE 
        WHEN Age < 30 THEN 'Under 30'
        WHEN Age BETWEEN 30 AND 40 THEN '30-40'
        WHEN Age BETWEEN 40 AND 50 THEN '40-50'
        WHEN Age > 50 THEN 'Over 50'
    END, Attrition
ORDER BY 
    CASE 
        WHEN Age < 30 THEN 'Under 30'
        WHEN Age BETWEEN 30 AND 40 THEN '30-40'
        WHEN Age BETWEEN 40 AND 50 THEN '40-50'
        WHEN Age > 50 THEN 'Over 50'
    END, Attrition DESC;
"""

# Execute query and display results
age_attrition = pd.read_sql_query(query_age, conn)
age_attrition


,row,Attrition,Age_group,count,percent_by_age
0,1,Yes,30-40,94,13.84
1,2,No,30-40,585,86.16
2,3,Yes,40-50,34,10.56
3,4,No,40-50,288,89.44
4,5,Yes,Over 50,18,12.59
5,6,No,Over 50,125,87.41
6,7,Yes,Under 30,91,27.91
7,8,No,Under 30,235,72.09


### Find Attrition by Monthly Income

In [26]:
# SQL query to find average income by department and job level, comparing employees who left vs stayed
query_income = """
SELECT 
    Department,
    JobLevel,
    ROUND(AVG(MonthlyIncome), 1) as avg_income,
    ROUND(AVG(CASE WHEN Attrition = 'Yes' THEN MonthlyIncome END), 1) as attrition_avg_income,
    ROUND(AVG(CASE WHEN Attrition = 'Yes' THEN MonthlyIncome END) - AVG(MonthlyIncome), 1) as difference
FROM employees
GROUP BY Department, JobLevel
HAVING COUNT(CASE WHEN Attrition = 'Yes' THEN 1 END) > 0
ORDER BY Department, JobLevel;
"""

# Execute query and display results
income_attrition = pd.read_sql_query(query_income, conn)
income_attrition
    

,Department,JobLevel,avg_income,attrition_avg_income,difference
0,Human Resources,1,2733.2,2415.7,-317.5
1,Human Resources,3,9623.0,10216.0,593.0
2,Research & Development,1,2840.1,2687.4,-152.7
3,Research & Development,2,5291.2,5372.0,80.8
4,Research & Development,3,10170.5,9503.8,-666.6
5,Research & Development,4,15634.7,12169.0,-3465.7
6,Research & Development,5,19218.5,19550.0,331.5
7,Sales,1,2506.7,2373.4,-133.3
8,Sales,2,5746.1,5917.0,170.9
9,Sales,3,9282.3,9202.8,-79.5


### Find Attrition by Years at Company


In [25]:
# SQL query to find attrition by years at company
query_tenure = """
SELECT 
    ROW_NUMBER() OVER (ORDER BY 
        CASE 
            WHEN YearsAtCompany < 2 THEN 1
            WHEN YearsAtCompany BETWEEN 2 AND 5 THEN 2
            WHEN YearsAtCompany BETWEEN 6 AND 10 THEN 3
            WHEN YearsAtCompany BETWEEN 11 AND 20 THEN 4
            WHEN YearsAtCompany > 20 THEN 5
        END
    ) as row,
    CASE 
        WHEN YearsAtCompany < 2 THEN 'New hires'
        WHEN YearsAtCompany BETWEEN 2 AND 5 THEN '2 - 5'
        WHEN YearsAtCompany BETWEEN 6 AND 10 THEN '6 - 10'
        WHEN YearsAtCompany BETWEEN 11 AND 20 THEN '11 - 20'
        WHEN YearsAtCompany > 20 THEN 'Over 20'
    END as tenure,
    SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) as count,
    ROUND(
        (SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) * 100.0 / (SELECT SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) FROM employees)), 2
    ) as percentage
FROM employees
GROUP BY 
    CASE 
        WHEN YearsAtCompany < 2 THEN 'New hires'
        WHEN YearsAtCompany BETWEEN 2 AND 5 THEN '2 - 5'
        WHEN YearsAtCompany BETWEEN 6 AND 10 THEN '6 - 10'
        WHEN YearsAtCompany BETWEEN 11 AND 20 THEN '11 - 20'
        WHEN YearsAtCompany > 20 THEN 'Over 20'
    END
ORDER BY 
    CASE 
        WHEN YearsAtCompany < 2 THEN 1
        WHEN YearsAtCompany BETWEEN 2 AND 5 THEN 2
        WHEN YearsAtCompany BETWEEN 6 AND 10 THEN 3
        WHEN YearsAtCompany BETWEEN 11 AND 20 THEN 4
        WHEN YearsAtCompany > 20 THEN 5
    END;
"""

# Execute query and display results
tenure_attrition = pd.read_sql_query(query_tenure, conn)
tenure_attrition

,row,tenure,count,percentage
0,1,New hires,75,31.65
1,2,2 - 5,87,36.71
2,3,6 - 10,55,23.21
3,4,11 - 20,12,5.06
4,5,Over 20,8,3.38


## 3. Explore Main Reasons for Attrition

### Question 1: Why do more people over 50 years old leave the company than people aged 40-50?

This analysis will compare the 40-50 and 50+ age groups to understand what factors might be driving higher attrition in the older demographic.


In [ ]:
# Compare attrition factors between 40-50 and 50+ age groups
query_age_comparison = """
SELECT 
    CASE 
        WHEN Age BETWEEN 40 AND 49 THEN '40-49 years'
        WHEN Age >= 50 THEN '50+ years'
    END as age_group,
    COUNT(*) as total_employees,
    ROUND(AVG(Age), 1) as avg_age,
    ROUND(
        (SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*)), 2
    ) as attrition_rate,
    ROUND(AVG(MonthlyIncome), 0) as avg_income,
    ROUND(AVG(WorkLifeBalance), 2) as avg_work_life_balance,
    ROUND(AVG(JobSatisfaction), 2) as avg_job_satisfaction,
    ROUND(AVG(YearsAtCompany), 1) as avg_years_at_company,
    ROUND(AVG(TotalWorkingYears), 1) as avg_total_experience,
    ROUND(
        (SUM(CASE WHEN OverTime = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*)), 2
    ) as overtime_percentage
FROM employees
WHERE Age >= 40
GROUP BY 
    CASE 
        WHEN Age BETWEEN 40 AND 49 THEN '40-49 years'
        WHEN Age >= 50 THEN '50+ years'
    END
ORDER BY age_group;
"""

age_comparison = pd.read_sql_query(query_age_comparison, conn)
print("=== COMPARISON: 40-49 vs 50+ AGE GROUPS ===")
print(age_comparison)

print(f"\nKey Insights:")
for idx, row in age_comparison.iterrows():
    print(f"\n{row['age_group']} Group:")
    print(f"  Attrition Rate: {row['attrition_rate']}%")
    print(f"  Average Income: ${row['avg_income']:,.0f}")
    print(f"  Work-Life Balance Score: {row['avg_work_life_balance']}/4")
    print(f"  Job Satisfaction Score: {row['avg_job_satisfaction']}/4")
    print(f"  Average Company Tenure: {row['avg_years_at_company']} years")
    print(f"  Work Experience: {row['avg_total_experience']} years")
    print(f"  Overtime Workers: {row['overtime_percentage']}%")


=== COMPARISON: 40-49 vs 50+ AGE GROUPS ===
     age_group  total_employees  avg_age  attrition_rate  avg_income  \
0  40-49 years              349     43.7            9.74      8538.0   
1    50+ years              173     53.8           13.29     10943.0   

   avg_work_life_balance  avg_job_satisfaction  avg_years_at_company  \
0                   2.81                  2.67                   8.7   
1                   2.65                  2.75                   9.9   

   avg_total_experience  overtime_percentage  
0                  15.1                29.23  
1                  21.6                32.95  

Key Insights:

40-49 years Group:
  • Attrition Rate: 9.74%
  • Average Income: $8,538
  • Work-Life Balance Score: 2.81/4
  • Job Satisfaction Score: 2.67/4
  • Average Company Tenure: 8.7 years
  • Total Work Experience: 15.1 years
  • Overtime Workers: 29.23%

50+ years Group:
  • Attrition Rate: 13.29%
  • Average Income: $10,943
  • Work-Life Balance Score: 2.65/4
  • Job 

### Question 2: Why do people with higher pay still leave the company?

Let's analyze high-income employees who left to understand what factors beyond compensation drive their attrition.


In [ ]:
# Analyze high-income employees who left the company
query_high_income_attrition = """
SELECT 
    COUNT(*) as high_income_left,
    ROUND(AVG(MonthlyIncome), 0) as avg_income,
    ROUND(AVG(WorkLifeBalance), 2) as avg_work_life_balance,
    ROUND(AVG(JobSatisfaction), 2) as avg_job_satisfaction,
    ROUND(AVG(EnvironmentSatisfaction), 2) as avg_env_satisfaction,
    ROUND(AVG(YearsAtCompany), 1) as avg_tenure,
    ROUND(AVG(DistanceFromHome), 1) as avg_distance_from_home,
    ROUND(
        (SUM(CASE WHEN OverTime = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*)), 2
    ) as overtime_percentage,
    ROUND(
        (SUM(CASE WHEN BusinessTravel = 'Travel_Frequently' THEN 1 ELSE 0 END) * 100.0 / COUNT(*)), 2
    ) as frequent_travel_percentage
FROM employees
WHERE MonthlyIncome > 8000 AND Attrition = 'Yes';
"""

# Compare with high-income employees who stayed
query_high_income_stayed = """
SELECT 
    COUNT(*) as high_income_stayed,
    ROUND(AVG(MonthlyIncome), 0) as avg_income,
    ROUND(AVG(WorkLifeBalance), 2) as avg_work_life_balance,
    ROUND(AVG(JobSatisfaction), 2) as avg_job_satisfaction,
    ROUND(AVG(EnvironmentSatisfaction), 2) as avg_env_satisfaction,
    ROUND(AVG(YearsAtCompany), 1) as avg_tenure,
    ROUND(AVG(DistanceFromHome), 1) as avg_distance_from_home,
    ROUND(
        (SUM(CASE WHEN OverTime = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*)), 2
    ) as overtime_percentage,
    ROUND(
        (SUM(CASE WHEN BusinessTravel = 'Travel_Frequently' THEN 1 ELSE 0 END) * 100.0 / COUNT(*)), 2
    ) as frequent_travel_percentage
FROM employees
WHERE MonthlyIncome > 8000 AND Attrition = 'No';
"""

high_income_left = pd.read_sql_query(query_high_income_attrition, conn)
high_income_stayed = pd.read_sql_query(query_high_income_stayed, conn)

print("=== HIGH-INCOME EMPLOYEES ANALYSIS (Monthly Income > $8,000) ===")
print("\nEmployees who LEFT:")
print(high_income_left)
print("\nEmployees who STAYED:")
print(high_income_stayed)

print(f"\nComparison Analysis:")
left = high_income_left.iloc[0]
stayed = high_income_stayed.iloc[0]

print(f"High-Income Employees who Left ({left['high_income_left']} employees):")
print(f"  Work-Life Balance: {left['avg_work_life_balance']}/4")
print(f"  Job Satisfaction: {left['avg_job_satisfaction']}/4")
print(f"  Environment Satisfaction: {left['avg_env_satisfaction']}/4")
print(f"  Overtime Workers: {left['overtime_percentage']}%")
print(f"  Frequent Travelers: {left['frequent_travel_percentage']}%")
print(f"  Average Tenure: {left['avg_tenure']} years")
print(f"  Average Distance from Home: {left['avg_distance_from_home']} miles")

print(f"\nHigh-Income Employees who Stayed ({stayed['high_income_stayed']} employees):")
print(f"  Work-Life Balance: {stayed['avg_work_life_balance']}/4")
print(f"  Job Satisfaction: {stayed['avg_job_satisfaction']}/4")
print(f"  Environment Satisfaction: {stayed['avg_env_satisfaction']}/4")
print(f"  Overtime Workers: {stayed['overtime_percentage']}%")
print(f"  Frequent Travelers: {stayed['frequent_travel_percentage']}%")
print(f"  Average Tenure: {stayed['avg_tenure']} years")
print(f"  Average Distance from Home: {stayed['avg_distance_from_home']} miles")


=== HIGH-INCOME EMPLOYEES ANALYSIS (Monthly Income > $8,000) ===

Employees who LEFT:
   high_income_left  avg_income  avg_work_life_balance  avg_job_satisfaction  \
0                40     11529.0                   2.68                  2.35   

   avg_env_satisfaction  avg_tenure  avg_distance_from_home  \
0                   2.3        11.8                    11.2   

   overtime_percentage  frequent_travel_percentage  
0                 50.0                        20.0  

Employees who STAYED:
   high_income_stayed  avg_income  avg_work_life_balance  \
0                 341     13538.0                    2.8   

   avg_job_satisfaction  avg_env_satisfaction  avg_tenure  \
0                  2.71                  2.76        11.4   

   avg_distance_from_home  overtime_percentage  frequent_travel_percentage  
0                     9.1                26.39                       17.89  

Comparison Analysis:
High-Income Employees who Left (40.0 employees):
  • Work-Life Balance: 2.68/

### Question 3: Which factors drive employees who work at company less than 5 years to leave?

Analyzing early-career attrition helps identify onboarding and early engagement issues that could be addressed to improve retention.


In [ ]:
# Analyze factors driving early-tenure attrition (< 5 years)
query_early_attrition = """
SELECT 
    'Left Company' as status,
    COUNT(*) as employee_count,
    ROUND(AVG(Age), 1) as avg_age,
    ROUND(AVG(MonthlyIncome), 0) as avg_income,
    ROUND(AVG(WorkLifeBalance), 2) as avg_work_life_balance,
    ROUND(AVG(JobSatisfaction), 2) as avg_job_satisfaction,
    ROUND(AVG(EnvironmentSatisfaction), 2) as avg_env_satisfaction,
    ROUND(AVG(YearsAtCompany), 1) as avg_tenure,
    ROUND(AVG(YearsSinceLastPromotion), 1) as avg_years_since_promotion,
    ROUND(AVG(TrainingTimesLastYear), 1) as avg_training_times,
    ROUND(
        (SUM(CASE WHEN OverTime = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*)), 2
    ) as overtime_percentage,
    ROUND(
        (SUM(CASE WHEN MaritalStatus = 'Single' THEN 1 ELSE 0 END) * 100.0 / COUNT(*)), 2
    ) as single_percentage
FROM employees
WHERE YearsAtCompany < 5 AND Attrition = 'Yes'

UNION ALL

SELECT 
    'Stayed at Company' as status,
    COUNT(*) as employee_count,
    ROUND(AVG(Age), 1) as avg_age,
    ROUND(AVG(MonthlyIncome), 0) as avg_income,
    ROUND(AVG(WorkLifeBalance), 2) as avg_work_life_balance,
    ROUND(AVG(JobSatisfaction), 2) as avg_job_satisfaction,
    ROUND(AVG(EnvironmentSatisfaction), 2) as avg_env_satisfaction,
    ROUND(AVG(YearsAtCompany), 1) as avg_tenure,
    ROUND(AVG(YearsSinceLastPromotion), 1) as avg_years_since_promotion,
    ROUND(AVG(TrainingTimesLastYear), 1) as avg_training_times,
    ROUND(
        (SUM(CASE WHEN OverTime = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*)), 2
    ) as overtime_percentage,
    ROUND(
        (SUM(CASE WHEN MaritalStatus = 'Single' THEN 1 ELSE 0 END) * 100.0 / COUNT(*)), 2
    ) as single_percentage
FROM employees
WHERE YearsAtCompany < 5 AND Attrition = 'No';
"""

early_tenure_analysis = pd.read_sql_query(query_early_attrition, conn)
print("=== EARLY-TENURE EMPLOYEES ANALYSIS (< 5 Years at Company) ===")
print(early_tenure_analysis)

print(f"\nDetailed Comparison:")
left_data = early_tenure_analysis[early_tenure_analysis['status'] == 'Left Company'].iloc[0]
stayed_data = early_tenure_analysis[early_tenure_analysis['status'] == 'Stayed at Company'].iloc[0]

print(f"\nEmployees who LEFT (< 5 years tenure):")
print(f"  Count: {left_data['employee_count']} employees")
print(f"  Average Age: {left_data['avg_age']} years")
print(f"  Average Income: ${left_data['avg_income']:,.0f}")
print(f"  Work-Life Balance: {left_data['avg_work_life_balance']}/4")
print(f"  Job Satisfaction: {left_data['avg_job_satisfaction']}/4")
print(f"  Environment Satisfaction: {left_data['avg_env_satisfaction']}/4")
print(f"  Years Since Last Promotion: {left_data['avg_years_since_promotion']}")
print(f"  Training Times per Year: {left_data['avg_training_times']}")
print(f"  Overtime Workers: {left_data['overtime_percentage']}%")
print(f"  Single Employees: {left_data['single_percentage']}%")

print(f"\nEmployees who STAYED (< 5 years tenure):")
print(f"  Count: {stayed_data['employee_count']} employees")
print(f"  Average Age: {stayed_data['avg_age']} years")
print(f"  Average Income: ${stayed_data['avg_income']:,.0f}")
print(f"  Work-Life Balance: {stayed_data['avg_work_life_balance']}/4")
print(f"  Job Satisfaction: {stayed_data['avg_job_satisfaction']}/4")
print(f"  Environment Satisfaction: {stayed_data['avg_env_satisfaction']}/4")
print(f"  Years Since Last Promotion: {stayed_data['avg_years_since_promotion']}")
print(f"  Training Times per Year: {stayed_data['avg_training_times']}")
print(f"  Overtime Workers: {stayed_data['overtime_percentage']}%")
print(f"  Single Employees: {stayed_data['single_percentage']}%")


=== EARLY-TENURE EMPLOYEES ANALYSIS (< 5 Years at Company) ===
              status  employee_count  avg_age  avg_income  \
0       Left Company             141     30.9      3525.0   
1  Stayed at Company             439     36.3      5298.0   

   avg_work_life_balance  avg_job_satisfaction  avg_env_satisfaction  \
0                   2.67                  2.45                  2.38   
1                   2.77                  2.77                  2.77   

   avg_tenure  avg_years_since_promotion  avg_training_times  \
0         1.8                        0.6                 2.6   
1         2.3                        0.7                 2.8   

   overtime_percentage  single_percentage  
0                53.19              52.48  
1                22.55              30.07  

Detailed Comparison:

Employees who LEFT (< 5 years tenure):
  • Count: 141 employees
  • Average Age: 30.9 years
  • Average Income: $3,525
  • Work-Life Balance: 2.67/4
  • Job Satisfaction: 2.45/4
  • Enviro

In [12]:
# Close database connection
conn.close()
print("Database connection closed successfully.")


Database connection closed successfully.
